# Kostenabrechnung Maximus
## Import Tanken

In [1]:
import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

In [2]:
tanken_df = pd.read_csv('Maximus.csv',
                         skiprows=6,
                         skipfooter=144,
                         engine='python',
                         index_col=1,
                         usecols=[0, 2, 6],
                         encoding='utf-8',
                         header=0,
                         names=['Tachostand (km)', 'Datum', 'Kosten'])
tanken_df.index = pd.to_datetime(tanken_df.index)
tanken_df['Subtyp'] = 'Tanken'
tanken_df.head()

,Tachostand (km),Kosten,Subtyp
Datum,,,
2015-04-01 18:59:00,76157,60.18,Tanken
2015-04-04 19:24:00,77118,71.20,Tanken
2015-04-13 18:02:00,78048,71.83,Tanken
2015-04-24 17:42:00,78980,72.46,Tanken
2015-04-30 11:00:00,79833,63.05,Tanken


# Import Unterhalt

In [3]:
unterhalt_df = pd.read_csv('Maximus.csv',
                           skiprows=232,
                           skipfooter=37,
                           engine='python',
                           usecols=[0, 1, 2, 3, 7],
                           index_col=1,
                           encoding='utf-8')
unterhalt_df.index = pd.to_datetime(unterhalt_df.index)
unterhalt_df.head()

,Beschreibung,Tachostand (km),Kosten,Subtyp
Datum,,,,
2015-02-22 00:00:00,Kredit,76020.0,-17000.0,Finanzierung
2015-02-23 00:00:00,Kaufbetrag,76020.0,16180.0,Finanzierung
2015-03-02 00:00:00,Kreditrate 0,76020.0,350.0,Finanzierung
2015-03-09 14:00:00,Anzahlung,76020.0,500.0,Finanzierung
2015-03-30 07:19:00,Garantie,76020.0,499.0,Finanzierung


## Agglomerierter Unterhalt

In [4]:
joined_df = pd.concat([unterhalt_df, tanken_df])
joined_df = joined_df.sort_index()
joined_df['Weg'] = joined_df['Tachostand (km)'].diff()
joined_df.loc[joined_df.Weg.isna(), ['Weg']] = 0
joined_df.loc[(joined_df.Subtyp == 'Wartung') |
              (joined_df.Subtyp == 'Verschleißteile') |
              (joined_df.Subtyp == 'Reparatur') |
              (joined_df.Subtyp == 'Pflege') |
              (joined_df.Subtyp == 'Unfallschaden') |
              (joined_df.Subtyp == 'Tuning'), ['Subtyp']] = 'Wartung'
joined_df = joined_df.reset_index()
joined_df

,Datum,Beschreibung,Tachostand (km),Kosten,Subtyp,Weg
0,2015-02-22 00:00:00,Kredit,76020.0,-17000.00,Finanzierung,0.0
1,2015-02-23 00:00:00,Kaufbetrag,76020.0,16180.00,Finanzierung,0.0
2,2015-03-02 00:00:00,Kreditrate 0,76020.0,350.00,Finanzierung,0.0
3,2015-03-09 14:00:00,Anzahlung,76020.0,500.00,Finanzierung,0.0
4,2015-03-30 07:19:00,Garantie,76020.0,499.00,Finanzierung,0.0
...,...,...,...,...,...,...
321,2021-11-16 08:08:00,NaN,252260.0,83.17,Tanken,260.0
322,2021-12-04 17:44:00,NaN,253194.0,92.06,Tanken,934.0
323,2021-12-18 11:44:00,Diagnose Steuerkette,253640.0,53.79,Wartung,446.0
324,2021-12-21 00:00:00,Abmeldung,253640.0,7.80,Steuer,0.0


In [5]:
subtypes = joined_df.Subtyp.unique()
subtypes

array(['Finanzierung', 'Wartung', 'Tanken', 'Steuer', 'Versicherung'],
      dtype=object)

In [41]:
freq='1M'
aggregated_df = pd.DataFrame(index=pd.date_range('2015-01-01', '2021-12-31', freq=freq))

for ii in range(len(subtypes)):
    aggregated_df.loc[aggregated_df.index[0],
                      subtypes[ii]] = joined_df.query('Datum <= "%s" & Subtyp == "%s"' % (aggregated_df.index[0], 
                                                                                          subtypes[ii])).Kosten.sum()

for idx in aggregated_df.index[1:]:
    for ii in range(len(subtypes)):
        aggregated_df.loc[idx,
                          subtypes[ii]] = joined_df.query('Datum <= "%s" & Subtyp == "%s"' % (idx, 
                                                                                              subtypes[ii])).Kosten.sum()
        
        
aggregated_df.iloc[1:, :] = aggregated_df.diff().iloc[1:, :]
aggregated_df.loc[aggregated_df.Finanzierung < 0, ['Finanzierung']] = 0

display(aggregated_df.sum())
aggregated_df = aggregated_df.rolling(12, min_periods=1).mean()
aggregated_df.sum()

Finanzierung    17902.36
Wartung         13794.01
Tanken          13461.30
Steuer           1640.68
Versicherung     4649.74
dtype: float64

Finanzierung    19473.417323
Wartung         13679.672152
Tanken          13109.260053
Steuer           1676.435123
Versicherung     4813.599196
dtype: float64

In [42]:
ax = aggregated_df.plot(kind='area', 
                   stacked=True,
                   figsize=(18,9),
                   grid=True,
                   ylabel='Betrag [€]',
                   title='Kosten Maximus');

#x_labels = aggregated_df.index.strftime('%b %Y');
#ax.set_xticklabels(x_labels);